In [1]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load delays_zurich_transport dataset
dataset = openml.datasets.get_dataset(537)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))


/home/smarton/anaconda3/envs/GradientBasedTreeLearning/lib/python3.11/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


Training set size: 13209
Validation set size: 3303
Test set size: 4128


In [3]:
X_train.head()

,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
16490,4.7069,27,1683.0,288.0,873.0,258.0,38.00,-120.97
6090,3.8750,15,6409.0,1363.0,3359.0,1267.0,34.10,-117.87
10973,2.8828,26,2312.0,525.0,1273.0,437.0,33.76,-117.85
17287,5.0000,34,1664.0,292.0,705.0,257.0,34.35,-119.74
12615,5.0371,25,4719.0,745.0,1857.0,739.0,38.50,-121.51


In [4]:
X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

low_cardinality_indices = []
high_cardinality_indices = []

categorical_feature_indices = []
for column_index in range(X_train.shape[1]):
    if categorical_indicator[column_index]:
        categorical_feature_indices.append(column_index)
        if len(X_train.iloc[:,column_index].unique()) < 10:
            low_cardinality_indices.append(X_train.columns[column_index])
        else:
            high_cardinality_indices.append(X_train.columns[column_index])

In [5]:
encoder = ce.OrdinalEncoder(cols=X_train.columns[categorical_feature_indices])
encoder.fit(X_train)
X_train = encoder.transform(X_train).astype(np.float64)
X_valid = encoder.transform(X_valid).astype(np.float64)
X_test = encoder.transform(X_test).astype(np.float64)

In [6]:
from GRANDE import GRANDE

params = {
        'depth': 5,
        'n_estimators': 2048,

        'learning_rate_weights': 0.005,
        'learning_rate_index': 0.01,
        'learning_rate_values': 0.01,
        'learning_rate_leaf': 0.01,

        'optimizer': 'SWA',
        'cosine_decay_steps': 0,

        'initializer': 'RandomNormal',

        'loss': 'mse',
        'focal_loss': False,
        'temperature': 0.0,

        'from_logits': True,
        'apply_class_balancing': True,

        'dropout': 0.0,

        'selected_variables': 0.8,
        'data_subset_fraction': 1.0,
}

args = {
    'epochs': 1_000,
    'early_stopping_epochs': 25,
    'batch_size': 64,

    'cat_idx': categorical_feature_indices,
    'objective': 'regression',
    
    'metrics': ['R2'], # F1, Accuracy, R2
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2023-11-03 13:52:02.303496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 13:52:03.742882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/smarton/anaconda3/envs/GradientBasedTreeLearning/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Epoch 1/1000


2023-11-03 13:52:06.488342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46692 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:e1:00.0, compute capability: 8.6
2023-11-03 13:52:10.884560: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x228f3870 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-03 13:52:10.884614: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-11-03 13:52:10.900538: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-03 13:52:12.116757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-11-03 13:52:13.302401: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XL

207/207 [==============================] - 14s 33ms/step - loss: 0.6192 - val_loss: 0.4396
Epoch 2/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.3426 - val_loss: 0.3150
Epoch 3/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2738 - val_loss: 0.2793
Epoch 4/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2467 - val_loss: 0.2576
Epoch 5/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2329 - val_loss: 0.2503
Epoch 6/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2241 - val_loss: 0.2427
Epoch 7/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2150 - val_loss: 0.2350
Epoch 8/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2088 - val_loss: 0.2315
Epoch 9/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.2032 - val_loss: 0.2251
Epoch 10/1000
207/207 [==============================] - 5s 22ms/step - loss: 0.1985 

In [7]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    #class_weights = class_weights/sum(class_weights)#    
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    #sample_weights = sample_weights/sum(class_weights)
    return sample_weights

In [8]:
median = X_train.median(axis=0)
X_train= X_train.fillna(median)
X_vali = X_valid.fillna(median)
X_test = X_test.fillna(median)

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

quantile_noise = 1e-4
quantile_train = np.copy(X_train.values).astype(np.float64)
np.random.seed(42)
stds = np.std(quantile_train, axis=0, keepdims=True)
noise_std = quantile_noise / np.maximum(stds, quantile_noise)
quantile_train += noise_std * np.random.randn(*quantile_train.shape)       

scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
scaler.fit(quantile_train)

X_train = scaler.transform(X_train.values.astype(np.float64))
X_valid = scaler.transform(X_valid.values.astype(np.float64))
X_test = scaler.transform(X_test.values.astype(np.float64))

In [9]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )

preds_xgb = model_xgb.predict(X_test)

[0]	validation_0-rmse:167446.86071
[1]	validation_0-rmse:123461.26842
[2]	validation_0-rmse:95129.73385
[3]	validation_0-rmse:77098.85021
[4]	validation_0-rmse:66940.51428
[5]	validation_0-rmse:61018.23280
[6]	validation_0-rmse:57755.65847
[7]	validation_0-rmse:55922.73300
[8]	validation_0-rmse:53891.27601
[9]	validation_0-rmse:52744.53056
[10]	validation_0-rmse:51993.21825
[11]	validation_0-rmse:51628.49327
[12]	validation_0-rmse:50403.30174
[13]	validation_0-rmse:50048.79694
[14]	validation_0-rmse:49807.93998
[15]	validation_0-rmse:49615.41251
[16]	validation_0-rmse:49072.91487
[17]	validation_0-rmse:48735.69840
[18]	validation_0-rmse:48679.98165
[19]	validation_0-rmse:48617.46640
[20]	validation_0-rmse:48570.59028
[21]	validation_0-rmse:48237.31781
[22]	validation_0-rmse:48123.18886
[23]	validation_0-rmse:48067.26865
[24]	validation_0-rmse:47982.04806
[25]	validation_0-rmse:47859.80117
[26]	validation_0-rmse:47734.11631
[27]	validation_0-rmse:47771.09401
[28]	validation_0-rmse:47692

In [10]:
encoder = ce.OrdinalEncoder(cols=X_train_raw.columns[categorical_feature_indices])
encoder.fit(X_train_raw)
X_train_raw = encoder.transform(X_train_raw)
X_valid_raw = encoder.transform(X_valid_raw)
X_test_raw = encoder.transform(X_test_raw)

median = X_train_raw.median(axis=0)
X_train_raw = X_train_raw.fillna(median)
X_valid_raw = X_valid_raw.fillna(median)
X_test_raw = X_test_raw.fillna(median)

In [11]:
from catboost import CatBoostRegressor, Pool

model_catboost = CatBoostRegressor(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

preds_catboost = model_catboost.predict(X_test_raw)

Learning rate set to 0.076361
0:	learn: 112413.6920108	test: 104827.1150993	best: 104827.1150993 (0)	total: 82.1ms	remaining: 1m 22s
1:	learn: 108432.7221618	test: 101135.9559217	best: 101135.9559217 (1)	total: 88.5ms	remaining: 44.2s
2:	learn: 104873.7852911	test: 97747.7089652	best: 97747.7089652 (2)	total: 94.1ms	remaining: 31.3s
3:	learn: 101662.2574261	test: 94696.0484905	best: 94696.0484905 (3)	total: 98.8ms	remaining: 24.6s
4:	learn: 98328.0535611	test: 91562.6624424	best: 91562.6624424 (4)	total: 104ms	remaining: 20.6s
5:	learn: 95521.8734728	test: 88943.6284296	best: 88943.6284296 (5)	total: 108ms	remaining: 17.9s
6:	learn: 92760.0099356	test: 86362.9039896	best: 86362.9039896 (6)	total: 113ms	remaining: 16.1s
7:	learn: 90270.6254827	test: 84022.9547964	best: 84022.9547964 (7)	total: 118ms	remaining: 14.6s
8:	learn: 88352.4862758	test: 82318.7282439	best: 82318.7282439 (8)	total: 123ms	remaining: 13.5s
9:	learn: 86182.7273301	test: 80336.0797330	best: 80336.0797330 (9)	total: 

In [12]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande))

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb))

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost))

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

MAE GRANDE: 34161.88154069767
R2 Score GRANDE: 0.8052513624070673


MAE XGB: 34090.61652131783
R2 Score XGB: 0.811496555862947


MAE CatBoost: 32114.8941375969
R2 Score CatBoost: 0.8256524499655538


